In [1]:
import os
import re
import glob
import os
import sys
import ast
import os.path as op
from collections import defaultdict
from copy import deepcopy
import copy

import pickle
import mne
import scipy
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn import set_config
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn import tree
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from sklearn.feature_selection import SelectKBest, SelectPercentile, f_regression, SelectFpr, SelectFdr, SequentialFeatureSelector
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn import datasets, linear_model

warnings.filterwarnings("ignore")

sns.set_theme(style="whitegrid", palette="deep")


ModuleNotFoundError: No module named 'seaborn'

In [2]:
#create dataframe from participants.tsv containing columns 1 and 2
df = pd.read_csv('../ds003944/participants.tsv', sep='\t', usecols=[0,1])
print(df)


   participant_id       type
0        sub-1448    Control
1        sub-1824  Psychosis
2        sub-1971    Control
3        sub-1983  Psychosis
4        sub-1989    Control
..            ...        ...
77      sub-2184A  Psychosis
78      sub-2193A    Control
79      sub-2214A    Control
80      sub-2217A  Psychosis
81      sub-2221A    Control

[82 rows x 2 columns]


In [3]:
#add mne object to dataframe from vmrk files within folder that name equals value in column 1
df['psyhosis_raw_data'] = [mne.io.read_raw_brainvision('../ds003944/'+str(i)+'/eeg/'+str(i)+'_task-Rest_eeg.vhdr', preload=True) for i in df['participant_id']]

Extracting parameters from ../ds003944/sub-1448/eeg/sub-1448_task-Rest_eeg.vhdr...
Setting channel info structure...
Reading 0 ... 305999  =      0.000 ...   305.999 secs...
Extracting parameters from ../ds003944/sub-1824/eeg/sub-1824_task-Rest_eeg.vhdr...
Setting channel info structure...
Reading 0 ... 300999  =      0.000 ...   300.999 secs...
Extracting parameters from ../ds003944/sub-1971/eeg/sub-1971_task-Rest_eeg.vhdr...
Setting channel info structure...
Reading 0 ... 300999  =      0.000 ...   300.999 secs...
Extracting parameters from ../ds003944/sub-1983/eeg/sub-1983_task-Rest_eeg.vhdr...
Setting channel info structure...
Reading 0 ... 300999  =      0.000 ...   300.999 secs...
Extracting parameters from ../ds003944/sub-1989/eeg/sub-1989_task-Rest_eeg.vhdr...
Setting channel info structure...
Reading 0 ... 309999  =      0.000 ...   309.999 secs...
Extracting parameters from ../ds003944/sub-1990/eeg/sub-1990_task-Rest_eeg.vhdr...
Setting channel info structure...
Reading 0 ...

In [28]:
dir_with_data = "preprocessed-data/"
# df.insert(3, "processed_frequencies", False)

FREQ_BANDS = {    "theta": [4.5, 8.5],
                  "alpha": [8.5, 11.5],}


def eeg_power_band(epochs, FREQ_BANDS):
    """EEG relative power band feature extraction.

    This function takes an ``mne.Epochs`` object and creates EEG features based
    on relative power in specific frequency bands that are compatible with
    scikit-learn.

    Parameters
    ----------
    epochs : Epochs
        The data.

    Returns
    -------
    X : numpy array of shape [n_samples, 5]
        Transformed data.
    """

    spectrum = epochs.compute_psd(picks='eeg', fmin=0.5, fmax=30.)
    psds, freqs = spectrum.get_data(return_freqs=True)
    # Normalize the PSDs
    psds /= np.sum(psds, axis=-1, keepdims=True)

    X = []
    for fmin, fmax in FREQ_BANDS.values():
        psds_band = psds[:, (freqs >= fmin) & (freqs < fmax)].mean(axis=-1)
        X.append(psds_band.reshape(len(psds), -1))

    return np.concatenate(X, axis=1)

    
''' uwaga! poniżej jest try i except, żeby nie zatrzymywało się na błędzie, że nie ma pliku dla jakiegoś uczestnika (części plików nie ma, dla tych uczestników dla których był za bardzo zaszumiony sygnał'''
''' żeby przetestować czy wszystko działa i nie wywala błędów już na etapie pierwszegu uczestnika trzeba zakomentować to try i except'''

for sub_id in df['participant_id'].values:
    try:
        mat_epochs_readed = mne.read_epochs(f"{dir_with_data}{sub_id}-epo.fif")
        evoked = mat_epochs_readed.average()
        evoked_channels = evoked.pick_channels(['F7', 'F5', 'F3', 'F1', 'F2', 'F4', 'F6', 'F8', 'P7', 'P5', 'P3', 'P1', 'P2', 'P4', 'P6', 'P8', 'O1', 'O2', 'C5', 'C3', 'C1', 'C2', 'C4', 'C6', 'T9', 'T7', 'T8', 'T10'])
        print(eeg_power_band(evoked_channels, FREQ_BANDS))
        ''' tak jak widać liczy się to co powinno i printuje na ekran, ale nie umiem tego zapisać do data frame'a - powinno iść do nowej kolumny, w tym wierszu dla odpowiedniego uczestnika (aktualne sub_id)'''
        '''poniżej moje nieudolne próby zrobienia tego, które wywalały błąd'''
        # df.loc[df['participant_id'] == sub_id, 'processed_frequencies'] = eeg_power_band(evoked_channels, FREQ_BANDS)
        #df['processed_frequencies'] = df['participant_id'].where(df['participant_id']==sub_id, eeg_power_band(evoked_channels, FREQ_BANDS))
    except:
        print("file for "+sub_id+" doesn't exist")
        
    

Reading d:\Nina\Dokumenty\Studia\Kogni\magisterka\I semestr\machine learning\eggML-project\preprocessed-data\sub-1448-epo.fif ...
    Found the data of interest:
        t =       0.00 ...   29999.00 ms
        0 CTF compensation matrices available
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Setting up band-pass filter from 4.5 - 12 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.50
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.50 Hz)
- Upper passband edge: 11.50 Hz
- Upper transition bandwidth: 2.88 Hz (-6 dB cutoff frequency: 12.94 Hz)
- Filter length: 1651 samples (1.651 sec)

    Using multitaper spectrum estimation with 7 DPSS windows
[[0.00448718 0.00326293]
 [0.00432667 0.00339549]
 [0.00441751 